In [65]:
#!/usr/bin/env python
import snowflake.connector
import time
import hashlib as hashlib
import pandas as pd
import numpy as np
import calendar
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5
###
# https://docs.snowflake.com/en/user-guide/python-connector-example.html
###
from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

In [66]:
# Create main connection context
con = snowflake.connector.connect(
    user='anuragvuppala',
    password='Snow@Flake44',
    account='vx24548.europe-west2.gcp' ,
    warehouse='crime_WH',
)

In [67]:
c = con.cursor()
c.execute("SELECT current_version()").fetchone()
    

('6.2.1',)

In [70]:
c.execute('USE ROLE SYSADMIN')
c.execute('CREATE DATABASE IF NOT EXISTS CRIME_DB_2021')


In [71]:
c.execute('USE DATABASE CRIME_DB_2021')
c.execute('USE SCHEMA PUBLIC')

In [72]:
from glob import glob
filenames = glob('raw_datasets/*/2021*.csv',recursive=True) 
print(len(filenames))

520


In [73]:
from clean_data import clean_dataset
for file in filenames:
    # Read the data files individually 
    data = pd.read_csv(file)
    
    print('reading file' ,file, 'done')
    
    #Cleaning the data base for null values and replacing the with corrresponing valuse and droping unwanted columns
    data=clean_dataset(data)
    print(file,' Cleaned!!!')
    p = 'processed_datasets/'+file[21:] 
    data.to_csv(p,index=False)
    print(file,' Saved!!!!!!!')
    
    
    

reading file raw_datasets/2021-09/2021-09-sussex-street.csv done
Empty cells filled with respective values
unnecessary Columns removed
raw_datasets/2021-09/2021-09-sussex-street.csv  Cleaned!!!
raw_datasets/2021-09/2021-09-sussex-street.csv  Saved!!!!!!!
reading file raw_datasets/2021-09/2021-09-metropolitan-street.csv done
Empty cells filled with respective values
unnecessary Columns removed
raw_datasets/2021-09/2021-09-metropolitan-street.csv  Cleaned!!!
raw_datasets/2021-09/2021-09-metropolitan-street.csv  Saved!!!!!!!
reading file raw_datasets/2021-09/2021-09-cleveland-street.csv done
Empty cells filled with respective values
unnecessary Columns removed
raw_datasets/2021-09/2021-09-cleveland-street.csv  Cleaned!!!
raw_datasets/2021-09/2021-09-cleveland-street.csv  Saved!!!!!!!
reading file raw_datasets/2021-09/2021-09-norfolk-street.csv done
Empty cells filled with respective values
unnecessary Columns removed
raw_datasets/2021-09/2021-09-norfolk-street.csv  Cleaned!!!
raw_datasets

In [75]:
c.execute('CREATE OR REPLACE TABLE "CRIME_DATA_2021" ("crime_id" STRING, "month" STRING, "reported_by" STRING, "fall_within" STRING, "longitude" FLOAT, "latitude" FLOAT, "location" STRING, "lsoa_code" STRING, "lsoa_name" STRING, "crime_type" STRING, "lsat_outcome" STRING)')

ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 1 at position 260 unexpected '<EOF>'.